<a href="https://colab.research.google.com/github/darwinyusef/visualizacionlimpiezadatosunir/blob/master/limpiezadeDatosUnir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas
!pip install numpy
!pip install seaborn
!pip install matplotlib
!pip install re

In [ ]:
import pandas as pd
import re
from datetime import datetime
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#df = pd.read_csv('/content/datos_ventas.csv')
df = pd.read_csv('/content/drive/MyDrive/UNIR/visualizacion/datos_ventas.csv')
df.dropna()
df.drop_duplicates()

In [ ]:
# @title Limpiar los emails con regex
df['Email'] = df['Email'].str.strip()  # Quitar espacios iniciales y finales
df['Email'] = df['Email'].apply(lambda x: re.sub(r'^[^a-zA-Z]+|[^a-zA-Z]+$', '', x))
df['Email'] = df['Email'].str.replace('*', '@', regex=False)
df['Email'] = df['Email'].str.replace('?', '@', regex=False)
df['Email'] = df['Email'].str.replace('!', '@', regex=False)
df['Email'] = df['Email'].str.replace('  ', '@', regex=False)

In [ ]:
# @title Filtra los simbolos internos *_?!  , para ver los cambios
filtered_df = df['Email'].str.contains(r'[\*_?!  ]', regex=True)
for index, value in enumerate(filtered_df):
  if value == True:
    print(index, value, df["Email"][index])

In [ ]:
# @title Filtra los simbolos internos .. para ver los cambios

filtered_df2 = df['Email'].str.contains(r'\.\.', regex=True)
filtered_df2
for index, value in enumerate(filtered_df2):
  if value == True:
    print(index, value, df["Email"][index])

In [ ]:
hoy = datetime.today().strftime('%Y-%m-%d')
df['Fecha_Venta'] = df['Fecha_Venta'].astype(str)

# Limpiar las fechas eliminando caracteres no válidos y reemplazando fechas inválidas por la fecha actual
df['Fecha_Venta'] = df['Fecha_Venta'].apply(lambda x: re.sub(r'[^0-9\-]', '', x) if re.match(r'\d{4}-\d{2}-\d{2}', x) else hoy)
df['Fecha_Venta']

In [ ]:
# Imprimir las fechas inválidas
for index, value in enumerate(df['Fecha_Venta']):
    if 'None' in value:
      print(f'Index: {index}, Value: {value}, Fecha_Venta: {df["Fecha_Venta"][index]}')

df['Producto'][42]

In [ ]:
def informationClean(info):
  # Convert the column to string type, handling potential NaNs
  info = info.astype(str).apply(lambda x: re.sub(r'^[*?\._]+|[*?\._]+$', '', x).strip())
  info = info.str.lstrip()
  info = info.str.rstrip()
  a = info.str.contains(r'\.\.', regex=True)
  b = info.str.contains(r'[\*_?!  ]', regex=True)
  # a filtra los .. | b filtra *_?!   | c obtiene la info final
  return [a, b, info]

In [ ]:
# Imprimir las Productos inválidos
for index, value in enumerate(informationClean(df['Producto'])[0]):
    if value == True:
      print(f'Index: {index}, Value: {value}, Producto: {df["Producto"][index]}')

In [ ]:
df['Producto'] = informationClean(df['Producto'])[2]
df['Direccion'] = informationClean(df['Direccion'])[2]
df['Categoria'] = informationClean(df['Categoria'])[2]
df['Metodo_Pago'] = informationClean(df['Metodo_Pago'])[2]
df['Cliente'] = informationClean(df['Cliente'])[2]
df['Estado'] = informationClean(df['Estado'])[2]

In [ ]:
df['Comentario'] = df['Comentario'].str.lstrip('\n\r')
df['Comentario'] = df['Comentario'].str.lstrip(' ')
df['Comentario'] = df['Comentario'].str.lstrip()
# df['Comentario'] = df['Comentario'].fillna('Ninguno')
# df['Comentario'] = df['Comentario'].str.replace('', 'Ninguno', regex=False)


In [ ]:
def limpiar_telefono(telefono):
    # Primero extraemos el indicativo en paréntesis y limpiamos todo lo demás
    telefono = re.sub(r'[^\d\(\)\-]', '', telefono)
    telefono = re.sub(r'[-]', '', telefono)
    telefono.rstrip()
    telefono.lstrip()
    if re.match(r'\(\d{3}\)', telefono):
        telefono = re.sub(r'\D', '', telefono)
        return f'({telefono[:3]}) {telefono[4:100]}'
    elif len(telefono) < 11:
        return f'{telefono[:100]}'
    elif len(telefono) > 11:
        return f'({telefono[:3]}) {telefono[4:100]}'
    else:
        telefono = re.sub(r'\D', '', telefono)
        return telefono

# eliminar mini caracteres
def limpiarminimos(telefono):
  if len(telefono) < 9:
    # Remove the regex=False keyword argument
    telefono = telefono.replace('(', '')
    telefono = telefono.replace(')', '')
    return telefono
  return telefono

# Limpiar todos los números de teléfono
df['Telefono'] = [limpiar_telefono(t) for t in df['Telefono']]
df['Telefono'] = [limpiarminimos(t) for t in df['Telefono']]
df['Telefono'][20:50]

In [ ]:
df.value_counts()
inicioFalsos = df.applymap(lambda x: not bool(x))
countFalsos = inicioFalsos.sum()
totalesFalsos = countFalsos.values.sum()
countFalsos

In [ ]:
df.describe().round(0)

In [ ]:
falsos = df.applymap(lambda x: not bool(x))
conteo_falsos_por_columna = falsos.sum()
total_falsos = falsos.values.sum()

print("\nConteo de valores falsos por columna:")
print(conteo_falsos_por_columna)
print(f"\nTotal de valores falsos en el DataFrame: {total_falsos}")

In [ ]:
# Detectar solo cadenas vacías o nulas y no valores con texto en los comentarios
for idx, valor in enumerate(df['Comentario']):
    if valor == '' or pd.isnull(valor):  # Detecta cadenas vacías o nulos
        print(f"Índice: {idx}, Valor falso: {repr(valor)}, Valor original: {df['Comentario'][idx]}")
        df.loc[idx, 'Comentario'] = 'Ninguno'

In [ ]:
# @title Corrección de lo anterior en la capa de comentario
df['Comentario'] = df['Comentario'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True).str.strip()
df['Comentario'] = df['Comentario'].str.replace("Productodefectuoso", "Producto defectuoso", regex=False)
df['Comentario'] = df['Comentario'].str.replace("Producto  defectuoso", "Producto defectuoso", regex=False)
df['Comentario'] = df['Comentario'].replace("", "Ninguno")
# Contar las ocurrencias de cada producto
comment_counts = df['Comentario'].value_counts().reset_index()
comment_counts.columns = ['Comentario', 'Count']
comment_counts

In [ ]:
# Asegurarte de que las categorías y productos estén en mayúsculas
df['Categoria'] = df['Categoria'].str.upper()
# Contar las ocurrencias de cada categoría
category_counts = df['Categoria'].value_counts().reset_index()
category_counts.columns = ['Categoria', 'Count']
category_counts

In [ ]:
# Eliminar detalles signos especiales en las columnas 'Estado' y 'Metodo_Pago'
df['Estado'] = df['Estado'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True).str.strip()
df['Estado'] = df['Estado'].str.replace("Encamino", "En camino", regex=False)
df['Estado'] = df['Estado'].str.replace("En  camino", "En camino", regex=False)
# Contar las ocurrencias de cada Estado
state_counts = df['Estado'].value_counts().reset_index()
state_counts.columns = ['Estado', 'Count']
state_counts

In [ ]:
# Eliminar detalles signos especiales en las columnas 'Estado' y 'Metodo_Pago'
df['Ciudad'] = df['Ciudad'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True).str.strip()
df['Ciudad'] = df['Ciudad'].str.replace("Encamino", "En camino", regex=False)
df['Ciudad'] = df['Ciudad'].str.replace("En  camino", "En camino", regex=False)
# Contar las ocurrencias de cada Estado
city_counts = df['Ciudad'].value_counts().reset_index()
city_counts.columns = ['Ciudad', 'Count']
# define la cantidad de elementos repetidos
city_counts.duplicated().head()

In [ ]:
df['Metodo_Pago'] = df['Metodo_Pago'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)
# Contar las ocurrencias de cada Metodo_Pago
method_counts = df['Metodo_Pago'].value_counts().reset_index()
method_counts.columns = ['Metodo_Pago', 'Count']
method_counts

In [ ]:
df['Producto'] = df['Producto'].str.upper()

# Contar las ocurrencias de cada producto
product_counts = df['Producto'].value_counts().reset_index()
product_counts.columns = ['Producto', 'Count']
product_counts

In [ ]:
# Verificar el resultado
df.head(40)

In [ ]:
# @title Precio_Unitario

from matplotlib import pyplot as plt
df['Precio_Unitario'].plot(kind='hist', bins=20, title='Precio_Unitario')
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
# Exportar el DataFrame a un archivo CSV
df.to_csv('/content/drive/MyDrive/UNIR/visualizacion/limpio_ventas_dashboard.csv', index=False)

print("Archivo CSV exportado exitosamente.")